In [201]:
import pandas as pd
import numpy as np

In [299]:
#import file
f = open("../scraped_examples/" + "buckleyChordsLyrics.txt","r")
lyrics = f.read()

In [300]:
lyrics = lyrics.replace("\\r\\n","£")

In [301]:
lyrics = "[ch]" + lyrics

In [302]:
lyric_sections = lyrics.split("££")
lyric_sections

['[ch]C[/ch] [ch]Am[/ch] [ch]C[/ch] [ch]Am[/ch]',
 "[Verse 1]£[tab]  [ch]C[/ch]                 [ch]Am[/ch]£I heard there was a secret chord[/tab]£[tab]     [ch]C[/ch]                   [ch]Am[/ch]£That David played and it pleased the Lord[/tab]£[tab]    [ch]F[/ch]                [ch]G[/ch]               [ch]C[/ch]        [ch]G[/ch]£But you don't really care for music, do you?[/tab]£[tab]        [ch]C[/ch]                  [ch]F[/ch]           [ch]G[/ch]£Well it goes like this the fourth, the fifth[/tab]£[tab]    [ch]Am[/ch]                 [ch]F[/ch]£The minor fall and the major lift[/tab]£[tab]    [ch]G[/ch]            [ch]E7[/ch]             [ch]Am[/ch]£The baffled king composing hallelujah[/tab]",
 '[Chorus]£[tab]     [ch]F[/ch]           [ch]Am[/ch]          [ch]F[/ch]           [ch]C[/ch]    [ch]G[/ch]   [ch]C[/ch]      [ch]Am[/ch] [ch]C[/ch] [ch]Am[/ch]£Hallelujah, hallelujah, hallelujah, hallelu-u-u-u-jah ....[/tab]',
 '[Verse 2]£[tab]           [ch]C[/ch]                      

In [303]:
tags = ["Intro", "Interlude", "Chorus", "Pre-Chorus", "Verse", "Outro"]

In [339]:
chord_list = []
lyric_list = []
section_list = []
i=-1
for section in lyric_sections:
    i+=1
    # print(lyric_sections[section])
    
    # each section can be 1: just chords, 2: tag then chord/lyric pairs, 
    # 3: no tag and one line of chords, 4: no tag & multiple lines of chords
    # chord/lyric pairs must have line breaks
    
    # check for pairs
    if section.find("[/tab]")!=-1:

        # find potential tag
        space_loc = section.find(" ")
        bracket_loc = section.find("]")
        tag_loc = min(space_loc, bracket_loc)
        tag = section[section.find("[")+1:tag_loc]
        # print(tag)

        # section has TAG
        if tag in tags:
            print(str(i) + ": TAG " + tag)

            # remove tag
            new_section = section[section.find("£")+1:]
            # print(new_section)
        
            # extract tab sections
            pair_list = new_section.split("[/tab]")
            for pair in pair_list:
                if len(pair)>0:
                    start_loc = pair.find("[tab]")+5
                    if start_loc == -1:
                        start_loc = 0
                    pound_loc = pair[start_loc:].find("£")
                    chords = pair[start_loc:pound_loc+start_loc]
                    lyrics = pair[pound_loc+start_loc+1:]
                    # print(pair)
                    # print(chords)
                    # print(lyrics)
                    chords = chords.replace("[ch]","")
                    chords = chords.replace("[/ch]","")
                    chord_list += [chords]
                    lyric_list += [lyrics]

        # section has NO TAG
        else:
            print(str(i) + ": no tag, multiple lines " + tag)
            if section.find("[tab]")!=-1: print("contains pairs")
            
    # section has no pairs 
    else: print(str(i) + ": no pairs")

print(chord_list)
print(lyric_list)

0: no pairs
1: TAG Verse
2: TAG Chorus
3: TAG Verse
4: TAG Chorus
5: TAG Verse
6: TAG Chorus
7: TAG Verse
8: TAG Chorus
9: TAG Verse
10: TAG Outro
['  C                 Am', '     C                   Am', '    F                G               C        G', '        C                  F           G', '    Am                 F', '    G            E7             Am', '     F           Am          F           C    G   C      Am C Am', '           C                        Am', '    C               Am', '    F              G             C            G', '    C               F       G', '    Am                        F', '    G                  E7            Am', '     F           Am          F           C    G   C      Am C Am', 'C               Am', '     C                       Am', '  F             G             C          G', '     C                     F      G', '    Am               F', '       G               E7          Am', '     F           Am          F           C    G   C      A

In [343]:
phrase = lyric_list[0]
phrase

'I heard there was a secret chord'

In [348]:
space_locs = [0] + [i for i in range(len(phrase)) if phrase[i]==" "] + [len(phrase)]

In [351]:
[phrase[space_locs[i]:space_locs[i+1]] for i in range(len(space_locs)-1)]

['I', ' heard', ' there', ' was', ' a', ' secret', ' chord']

In [352]:
[chord_list[0][space_locs[i]:space_locs[i+1]] for i in range(len(space_locs)-1)]

[' ', ' C    ', '      ', '    ', '  ', ' Am', '']